# 🤝 उद्यम बहु-एजेन्ट कार्यप्रवाह प्रणाली (.NET)

## 📋 सिक्ने उद्देश्यहरू

यो नोटबुकले Microsoft Agent Framework प्रयोग गरेर .NET मा GitHub Models को साथमा परिष्कृत उद्यम-स्तरीय बहु-एजेन्ट प्रणाली निर्माण गर्ने तरिका प्रदर्शन गर्दछ। तपाईंले संरचित कार्यप्रवाहहरू मार्फत एकसाथ काम गर्ने विभिन्न विशेषज्ञ एजेन्टहरूलाई समन्वय गर्न सिक्नुहुनेछ, उत्पादन-तयार समाधानहरूको लागि .NET को उद्यम सुविधाहरूको उपयोग गर्दै।

**उद्यम बहु-एजेन्ट क्षमताहरू जुन तपाईं निर्माण गर्नुहुनेछ:**
- 👥 **एजेन्ट सहयोग**: टाइप-सुरक्षित एजेन्ट समन्वय कम्पाइल-समय मान्यतासहित
- 🔄 **कार्यप्रवाह समन्वय**: .NET को असिन्क ढाँचाहरूको साथ घोषणात्मक कार्यप्रवाह परिभाषा
- 🎭 **भूमिका विशेषज्ञता**: बलियो-टाइप गरिएको एजेन्ट व्यक्तित्व र विशेषज्ञता क्षेत्रहरू
- 🏢 **उद्यम एकीकरण**: निगरानी र त्रुटि ह्यान्डलिङको साथ उत्पादन-तयार ढाँचाहरू


## ⚙️ पूर्वापेक्षाहरू र सेटअप

**विकास वातावरण:**
- .NET 9.0 SDK वा उच्च संस्करण
- Visual Studio 2022 वा VS Code C# एक्सटेन्सनको साथ
- Azure सदस्यता (स्थायी एजेन्टहरूको लागि)

**आवश्यक NuGet प्याकेजहरू:**
```xml
<PackageReference Include="Microsoft.Extensions.AI.Abstractions" Version="9.9.0" />
<PackageReference Include="Azure.AI.Agents.Persistent" Version="1.2.0-beta.4" />
<PackageReference Include="Azure.Identity" Version="1.15.0" />
<PackageReference Include="System.Linq.Async" Version="6.0.3" />
<PackageReference Include="Microsoft.Extensions.AI" Version="9.8.0" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
```


In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI Abstractions
// Core AI abstraction layer providing standardized interfaces for multi-agent systems
// Essential for building type-safe, enterprise-grade agent orchestration
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.9.1"

Installed Packages Microsoft.Extensions.AI.Abstractions, 9.9.1

In [2]:
// 📦 Enterprise Multi-Agent Dependencies
// Azure.AI.Agents.Persistent: Enterprise agent management with state persistence and Azure integration
// Azure.Identity: Secure authentication for Azure services in multi-agent workflows
// System.Linq.Async: High-performance asynchronous LINQ operations for agent data processing
// Microsoft.Extensions.AI: Core AI service abstractions for agent coordination
// DotNetEnv: Secure environment variable management for multi-agent configuration
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.4"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: Microsoft.Extensions.AI, 9.9.1"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: OpenTelemetry.Api, 1.0.0"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.4 Azure.Identity, 1.15.0 DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.9.1 OpenTelemetry.Api, 1.0.1 System.Linq.Async, 6.0.3

In [3]:

#r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3

In [4]:
// 🔗 Local Assembly Reference: Microsoft Agents AI OpenAI
// OpenAI-compatible integration for GitHub Models in multi-agent systems
// Enables high-performance language model access for agent communication and reasoning
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [5]:
// 🔄 Local Assembly Reference: Microsoft Agents Workflows
// Advanced workflow orchestration engine for complex multi-agent coordination
// Provides declarative workflow definition and type-safe agent interaction patterns
// #r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.2"

Error: Microsoft.Agents.AI version 1.0.0-preview.251001.2 cannot be added because version 1.0.0-preview.251001.3 was added previously.

In [6]:
// 🔌 NuGet Package Reference: OpenAI Integration (Preview)
// Preview version providing GitHub Models compatibility for multi-agent workflows
// Enables seamless integration with GitHub's model inference service for agent communication
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;
using OpenAI;

In [9]:
 using DotNetEnv;

In [10]:
Env.Load("../../../.env");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [12]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [13]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [14]:
const string REVIEWER_NAME = "Concierge";
const string REVIEWER_INSTRUCTIONS = @"""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """;

In [15]:
const string FRONTDESK_NAME = "FrontDesk";
const string FRONTDESK_INSTRUCTIONS = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

In [16]:

ChatClientAgentOptions frontdeskAgentOptions = new(name: FRONTDESK_NAME, instructions: FRONTDESK_INSTRUCTIONS);
ChatClientAgentOptions reviewerAgentOptions = new(name: REVIEWER_NAME, instructions: REVIEWER_INSTRUCTIONS);

In [17]:
AIAgent reviewerAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    reviewerAgentOptions);
AIAgent frontdeskAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    frontdeskAgentOptions);

In [18]:
var workflow = new WorkflowBuilder(frontdeskAgent)
            .AddEdge(frontdeskAgent, reviewerAgent)
            .Build();

In [19]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, new ChatMessage(ChatRole.User, "I would like to go to Paris."));

In [20]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));

In [21]:
string strResult = "";

In [22]:
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
            if (evt is AgentRunUpdateEvent executorComplete)
            {
                strResult += executorComplete.Data;
                Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
            }
}

83b7c6d5c52d4dd39d39a753a9fe1d4a: 
83b7c6d5c52d4dd39d39a753a9fe1d4a: Visit
83b7c6d5c52d4dd39d39a753a9fe1d4a:  the
83b7c6d5c52d4dd39d39a753a9fe1d4a:  Eiffel
83b7c6d5c52d4dd39d39a753a9fe1d4a:  Tower
83b7c6d5c52d4dd39d39a753a9fe1d4a:  for
83b7c6d5c52d4dd39d39a753a9fe1d4a:  iconic
83b7c6d5c52d4dd39d39a753a9fe1d4a:  views
83b7c6d5c52d4dd39d39a753a9fe1d4a:  of
83b7c6d5c52d4dd39d39a753a9fe1d4a:  the
83b7c6d5c52d4dd39d39a753a9fe1d4a:  city
83b7c6d5c52d4dd39d39a753a9fe1d4a: .
83b7c6d5c52d4dd39d39a753a9fe1d4a: 
83b7c6d5c52d4dd39d39a753a9fe1d4a: 
63600cef458c48b0a1e525bde5511110: 
63600cef458c48b0a1e525bde5511110: This
63600cef458c48b0a1e525bde5511110:  recommendation
63600cef458c48b0a1e525bde5511110:  is
63600cef458c48b0a1e525bde5511110:  not
63600cef458c48b0a1e525bde5511110:  approved
63600cef458c48b0a1e525bde5511110: .
63600cef458c48b0a1e525bde5511110:  While
63600cef458c48b0a1e525bde5511110:  the
63600cef458c48b0a1e525bde5511110:  Eiffel
63600cef458c48b0a1e525bde5511110:  Tower
63600cef458c48

In [23]:
strResult

Visit the Eiffel Tower for iconic views of the city.This recommendation is not approved. While the Eiffel Tower is a must-see for many travelers, it is often crowded and heavily tourist-focused. To refine the recommendation, consider suggesting an experience that immerses travelers in the local culture, perhaps by exploring lesser-known neighborhoods, local markets, or hidden gems that showcase the authentic lifestyle of the city.


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी यथासम्भव शुद्धता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। मूल दस्तावेज़ यसको मातृभाषामा आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
